# Connect Four RL - Training with Live Metrics

Этот ноутбук позволяет:
- Настроить параметры обучения
- Запустить обучение DQN или Q-learning агента
- Визуализировать метрики в реальном времени


In [ ]:
# Автоматическая перезагрузка модулей при изменении (autoreload)
# Это позволяет изменять код в .py файлах без перезапуска ядра ноутбука
%load_ext autoreload
%autoreload 2

# Импорты
import sys
from pathlib import Path
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output, display
import threading
from collections import defaultdict

# Добавляем путь к проекту
sys.path.insert(0, str(Path().resolve().parent))

from src.envs import Connect4Env, RewardConfig
from src.agents import DQNAgent, QLearningAgent, RandomAgent, HeuristicAgent, SmartHeuristicAgent
from src.utils import MetricsLogger
from src.selfplay import SelfPlayConfig
from src.training.random_opening import RandomOpeningConfig
from src.training.train_dqn import train_dqn
from src.training.train_qlearning import train_qlearning

print("Импорты успешно загружены!")


## Настройка параметров обучения

Выберите параметры для обучения:


In [ ]:
# Параметры обучения
TRAINING_CONFIG = {
    # Тип модели
    "model_type": "dqn",  # "dqn" или "qlearning"
    "network_type": "dueling_dqn", 
    
    
    # Параметры обучения
    "num_episodes": 100000,
    "learning_rate": 0.0001,
    "discount_factor": 0.99,
    "epsilon": 1.0,
    "epsilon_decay": 0.9997,  # Увеличено с 0.995 для более медленного decay
    # 0.995 -> минимум на эпизоде ~919 (слишком быстро)
    # 0.999 -> минимум на эпизоде ~4603
    # 0.9995 -> минимум на эпизоде ~9209 (рекомендуется для 50000 эпизодов)
    # 0.9999 -> минимум на эпизоде ~46050 (очень медленно)
    "epsilon_min": 0.03,
    
    # DQN специфичные параметры (игнорируются для Q-learning)
    "batch_size": 32,
    "replay_buffer_size": 20000,
    "target_update_freq": 100,
    "soft_update": True,
    "tau": 0.01,
    
    # Оценка и сохранение
    "eval_freq": 100,
    "eval_episodes": 100,
    "save_freq": 1000,
    
    # Пути (используем абсолютные пути относительно корня проекта)
    "checkpoint_dir": str(Path().resolve().parent / "data" / "checkpoints"),
    "log_dir": str(Path().resolve().parent / "data" / "logs"),
    
    # Другие
    "device": None,  # None для auto-detect, или "cuda" или "cpu"
    "seed": 42,
}

# Настройка наград
REWARD_CONFIG = RewardConfig(
    win=1.0,  # Награда за победу
    loss=-1.0,  # Награда за поражение
    draw=0.0,  # Награда за ничью
    three_in_row=0.00,  # Награда за 3 в ряд (промежуточная награда)
    opponent_three_in_row=-0.00,  # Штраф за вражеские 3 в ряд (промежуточный штраф)
    invalid_action=-0.1,  # Награда за неверный ход
)

# Распределение эвристических оппонентов
HEURISTIC_DISTRIBUTION = {
    "random": 0.1,
    "heuristic": 0.7,
    "smart_heuristic": 0.2,
}

# Конфигурация self-play (обучение против старых версий себя)
# Установите SELF_PLAY_CONFIG = None, чтобы отключить self-play
SELF_PLAY_CONFIG = SelfPlayConfig(
    start_episode=5000,  # С какого эпизода начинать использовать frozen DQN
    save_every=500,  # Как часто добавлять новую "версию себя" в пул (может отличаться от save_freq)
    max_frozen_agents=20,  # Сколько старых версий максимум держать в пуле
    fraction=0.5,  # Доля эпизодов, где оппонент = frozen DQN (0.0-1.0)
)

# Конфигурация случайных стартовых позиций (пролог перед обучением агента)
# Установите RANDOM_OPENING_CONFIG = None, чтобы отключить случайный пролог
RANDOM_OPENING_CONFIG = RandomOpeningConfig(
    probability=0.5,  # Вероятность запуска случайного пролога перед эпизодом
    min_half_moves=2,  # Минимальное количество полуходов случайного пролога
    max_half_moves=6,  # Максимальное количество полуходов случайного пролога
)

# Параметры визуализации
PLOT_CONFIG = {
    "update_freq": 50,  # Обновлять графики каждые N эпизодов
    "metrics_to_plot": [
        "win_rate",
        "loss_rate",
        "episode_reward",
        "episode_length",
        "epsilon",
        "train_loss",
        "train_avg_q",
        "train_td_error",
    ],
}

print("Параметры настроены!")
print(f"Тип модели: {TRAINING_CONFIG['model_type']}")
print(f"Количество эпизодов: {TRAINING_CONFIG['num_episodes']}")
print(f"\nНаграды:")
print(f"  Победа: {REWARD_CONFIG.win}")
print(f"  Поражение: {REWARD_CONFIG.loss}")
print(f"  Ничья: {REWARD_CONFIG.draw}")
print(f"  3 в ряд: {REWARD_CONFIG.three_in_row}")
print(f"  Вражеские 3 в ряд (штраф): {REWARD_CONFIG.opponent_three_in_row}")
print(f"  Неверный ход: {REWARD_CONFIG.invalid_action}")
print(f"\nРаспределение эвристик: {HEURISTIC_DISTRIBUTION}")
print("\nSelf-play:")
if SELF_PLAY_CONFIG is None:
    print("  Статус: отключен")
else:
    print("  Статус: включен")
    print(f"  Начало с эпизода: {SELF_PLAY_CONFIG.start_episode}")
    print(f"  Доля self-play: {SELF_PLAY_CONFIG.fraction:.1%}")
    print(f"  Сохранение каждые: {SELF_PLAY_CONFIG.save_every} эпизодов")
    print(f"  Макс. frozen агентов: {SELF_PLAY_CONFIG.max_frozen_agents}")

print("\nRandom opening:")
if RANDOM_OPENING_CONFIG is None:
    print("  Статус: отключен")
else:
    print("  Статус: включен")
    print(f"  Вероятность: {RANDOM_OPENING_CONFIG.probability:.2f}")
    print(f"  Полуходы: {RANDOM_OPENING_CONFIG.min_half_moves}-{RANDOM_OPENING_CONFIG.max_half_moves}")


## Функции для визуализации метрик


In [ ]:
def load_metrics_from_csv(log_dir: str, metrics_file: str = None) -> pd.DataFrame:
    """Загрузить метрики из CSV файла.
    
    Args:
        log_dir: Директория с логами
        metrics_file: Путь к конкретному файлу метрик (если None, берется последний по времени создания)
    """
    if metrics_file and Path(metrics_file).exists():
        # Используем указанный файл
        df = pd.read_csv(metrics_file)
        return df
    
    # Ищем последний файл по времени создания (не модификации!)
    csv_files = list(Path(log_dir).glob("metrics_*.csv"))
    if not csv_files:
        return pd.DataFrame()
    
    # Берем последний файл по времени создания (st_ctime), а не модификации
    # Это более надежно, так как файл создается один раз при запуске обучения
    latest_file = max(csv_files, key=lambda x: x.stat().st_ctime)
    df = pd.read_csv(latest_file)
    return df

def plot_metrics(df: pd.DataFrame, metrics_to_plot: list, figsize=(15, 10)):
    """Построить графики метрик."""
    if df.empty:
        print("Нет данных для построения графиков")
        return
    
    n_metrics = len(metrics_to_plot)
    n_cols = 2
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    if n_metrics == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    for i, metric in enumerate(metrics_to_plot):
        if metric not in df.columns:
            continue
        
        ax = axes[i]
        ax.plot(df['step'], df[metric], label=metric, linewidth=2)
        ax.set_xlabel('Episode')
        ax.set_ylabel(metric)
        ax.set_title(f'{metric} over time')
        ax.grid(True, alpha=0.3)
        ax.legend()
        
        # Добавляем последнее значение
        if len(df) > 0:
            last_val = df[metric].iloc[-1]
            ax.axhline(y=last_val, color='r', linestyle='--', alpha=0.5)
            ax.text(len(df), last_val, f'{last_val:.3f}', 
                   verticalalignment='bottom', fontsize=9)
    
    # Скрываем лишние subplots
    for i in range(n_metrics, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

def plot_win_rate_comparison(df: pd.DataFrame):
    """Построить график сравнения win/draw/loss rates."""
    if df.empty or 'win_rate' not in df.columns:
        return
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    if 'win_rate' in df.columns:
        ax.plot(df['step'], df['win_rate'], label='Win Rate', color='green', linewidth=2)
    if 'draw_rate' in df.columns:
        ax.plot(df['step'], df['draw_rate'], label='Draw Rate', color='gray', linewidth=2)
    if 'loss_rate' in df.columns:
        ax.plot(df['step'], df['loss_rate'], label='Loss Rate', color='red', linewidth=2)
    
    ax.set_xlabel('Episode')
    ax.set_ylabel('Rate')
    ax.set_title('Win/Draw/Loss Rates over Time')
    ax.set_ylim([0, 1])
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("Функции визуализации готовы!")


## Запуск обучения с визуализацией

Запустите обучение. Графики будут обновляться автоматически.


In [ ]:
# Создаем уникальную папку для этого запуска обучения
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_type = TRAINING_CONFIG['model_type']
run_id = f"{model_type}_{timestamp}"

# Создаем подпапки для этого запуска (глобальные переменные для использования в других ячейках)
run_checkpoint_dir = os.path.join(TRAINING_CONFIG['checkpoint_dir'], run_id)
run_log_dir = os.path.join(TRAINING_CONFIG['log_dir'], run_id)

# Создаем директории
os.makedirs(run_checkpoint_dir, exist_ok=True)
os.makedirs(run_log_dir, exist_ok=True)

print(f"📁 Создана папка для этого запуска обучения:")
print(f"   Checkpoints: {run_checkpoint_dir}")
print(f"   Logs: {run_log_dir}")

# Запускаем обучение в отдельном потоке
training_complete = threading.Event()
stop_training_flag = threading.Event()  # Флаг для остановки обучения
training_thread = None
current_metrics_file = None  # Путь к текущему файлу метрик

def run_training():
    """Запустить обучение в отдельном потоке."""
    global current_metrics_file
    try:
        # Определяем путь к файлу метрик, который будет создан
        # MetricsLogger создает файл с timestamp при инициализации
        current_metrics_file = os.path.join(run_log_dir, f"metrics_{timestamp}.csv")
        if TRAINING_CONFIG['model_type'] == 'dqn':
            train_dqn(
                num_episodes=TRAINING_CONFIG['num_episodes'],
                learning_rate=TRAINING_CONFIG['learning_rate'],
                discount_factor=TRAINING_CONFIG['discount_factor'],
                epsilon=TRAINING_CONFIG['epsilon'],
                epsilon_decay=TRAINING_CONFIG['epsilon_decay'],
                epsilon_min=TRAINING_CONFIG['epsilon_min'],
                batch_size=TRAINING_CONFIG['batch_size'],
                replay_buffer_size=TRAINING_CONFIG['replay_buffer_size'],
                target_update_freq=TRAINING_CONFIG['target_update_freq'],
                soft_update=TRAINING_CONFIG['soft_update'],
                tau=TRAINING_CONFIG['tau'],
                eval_freq=TRAINING_CONFIG['eval_freq'],
                eval_episodes=TRAINING_CONFIG['eval_episodes'],
                save_freq=TRAINING_CONFIG['save_freq'],
                checkpoint_dir=run_checkpoint_dir,
                log_dir=run_log_dir,
                device=TRAINING_CONFIG['device'],
                seed=TRAINING_CONFIG['seed'],
                stop_flag=stop_training_flag,
                reward_config=REWARD_CONFIG,
                heuristic_distribution=HEURISTIC_DISTRIBUTION,
                self_play_config=SELF_PLAY_CONFIG,
                network_type=TRAINING_CONFIG['network_type'],
                random_opening_config=RANDOM_OPENING_CONFIG,
            )
        elif TRAINING_CONFIG['model_type'] == 'qlearning':
            train_qlearning(
                num_episodes=TRAINING_CONFIG['num_episodes'],
                learning_rate=TRAINING_CONFIG['learning_rate'],
                discount_factor=TRAINING_CONFIG['discount_factor'],
                epsilon=TRAINING_CONFIG['epsilon'],
                epsilon_decay=TRAINING_CONFIG['epsilon_decay'],
                epsilon_min=TRAINING_CONFIG['epsilon_min'],
                eval_freq=TRAINING_CONFIG['eval_freq'],
                eval_episodes=TRAINING_CONFIG['eval_episodes'],
                save_freq=TRAINING_CONFIG['save_freq'],
                checkpoint_dir=run_checkpoint_dir,
                log_dir=run_log_dir,
                seed=TRAINING_CONFIG['seed'],
                stop_flag=stop_training_flag,
                reward_config=REWARD_CONFIG,
                random_opening_config=RANDOM_OPENING_CONFIG,
            )
    except Exception as e:
        print(f"Ошибка при обучении: {e}")
        import traceback
        traceback.print_exc()
    finally:
        training_complete.set()

# Запускаем обучение
print("Запуск обучения...")
training_thread = threading.Thread(target=run_training, daemon=True)
training_thread.start()

print("Обучение запущено в фоновом режиме!")
print("Используйте следующую ячейку для мониторинга прогресса.")


## Остановка обучения

**Используйте эту ячейку для остановки обучения, если оно запущено в фоновом режиме.**


In [ ]:
# Остановка обучения
if 'stop_training_flag' in globals() and stop_training_flag is not None:
    stop_training_flag.set()
    print("🛑 Сигнал остановки отправлен обучению!")
    print("Обучение остановится после завершения текущего эпизода.")
    print("Проверьте метрики через несколько секунд.")
else:
    print("⚠️  Обучение не запущено или переменные не инициализированы.")
    print("Если обучение все еще работает, попробуйте:")
    print("1. Перезапустить ядро ноутбука (Kernel -> Restart)")
    print("2. Или найти процесс Python и остановить его вручную:")
    print("   ps aux | grep train_dqn")
    print("   kill <PID>")


## Мониторинг обучения в реальном времени

Запустите эту ячейку для обновления графиков. Можно запускать многократно.


In [ ]:
# Загружаем метрики (используем текущий файл, если он определен)
# Используем run_log_dir, если он определен, иначе fallback на TRAINING_CONFIG['log_dir']
log_dir_to_use = run_log_dir if 'run_log_dir' in globals() else TRAINING_CONFIG['log_dir']

if 'current_metrics_file' in globals() and current_metrics_file and Path(current_metrics_file).exists():
    df = load_metrics_from_csv(log_dir_to_use, metrics_file=current_metrics_file)
    print(f"📊 Используется файл: {Path(current_metrics_file).name}")
else:
    df = load_metrics_from_csv(log_dir_to_use)
    if not df.empty:
        # Показываем, какой файл был загружен
        csv_files = list(Path(log_dir_to_use).glob("metrics_*.csv"))
        if csv_files:
            latest_file = max(csv_files, key=lambda x: x.stat().st_ctime)
            print(f"📊 Используется файл: {latest_file.name}")

if not df.empty:
    print(f"Загружено {len(df)} записей метрик")
    print(f"Последний эпизод: {df['step'].max()}")
    print(f"\nПоследние метрики:")
    print(df.tail(1)[['step'] + [m for m in PLOT_CONFIG['metrics_to_plot'] if m in df.columns]].to_string())
    print()
    
    # Строим графики
    clear_output(wait=True)
    
    # График win/draw/loss rates
    plot_win_rate_comparison(df)
    
    # Графики других метрик
    plot_metrics(df, PLOT_CONFIG['metrics_to_plot'])
    
    # Проверяем, завершено ли обучение
    if training_complete.is_set():
        print("✅ Обучение завершено!")
    else:
        print(f"⏳ Обучение продолжается... (эпизод {df['step'].max()}/{TRAINING_CONFIG['num_episodes']})")
else:
    print("⏳ Ожидание данных... Обучение только началось.")
    if training_complete.is_set():
        print("✅ Обучение завершено, но метрики еще не загружены.")


## Автоматическое обновление графиков (опционально)

Запустите эту ячейку для автоматического обновления графиков каждые N секунд.


In [ ]:
# Автоматическое обновление графиков
import time
from IPython.display import clear_output

UPDATE_INTERVAL = 10  # Обновлять каждые N секунд
MAX_UPDATES = 1000  # Максимальное количество обновлений

print(f"Начинаю автоматическое обновление графиков каждые {UPDATE_INTERVAL} секунд...")
print("Нажмите Interrupt (Ctrl+C) чтобы остановить")

for update_count in range(MAX_UPDATES):
    # Загружаем метрики (используем текущий файл, если он определен)
    # Используем run_log_dir, если он определен, иначе fallback на TRAINING_CONFIG['log_dir']
    log_dir_to_use = run_log_dir if 'run_log_dir' in globals() else TRAINING_CONFIG['log_dir']
    
    if 'current_metrics_file' in globals() and current_metrics_file and Path(current_metrics_file).exists():
        df = load_metrics_from_csv(log_dir_to_use, metrics_file=current_metrics_file)
    else:
        df = load_metrics_from_csv(log_dir_to_use)
    
    if not df.empty:
        clear_output(wait=True)
        
        print(f"Обновление #{update_count + 1} | Эпизод: {df['step'].max()}/{TRAINING_CONFIG['num_episodes']}")
        
        # График win/draw/loss rates
        plot_win_rate_comparison(df)
        
        # Графики других метрик
        plot_metrics(df, PLOT_CONFIG['metrics_to_plot'])
        
        # Проверяем, завершено ли обучение
        if training_complete.is_set():
            print("✅ Обучение завершено!")
            break
        
        # Показываем последние метрики
        if len(df) > 0:
            last_row = df.iloc[-1]
            print(f"\nПоследние метрики (эпизод {last_row['step']}):")
            for metric in PLOT_CONFIG['metrics_to_plot']:
                if metric in last_row:
                    print(f"  {metric}: {last_row[metric]:.4f}")
    else:
        print(f"⏳ Ожидание данных... ({update_count + 1}/{MAX_UPDATES})")
    
    # Ждем перед следующим обновлением
    time.sleep(UPDATE_INTERVAL)


## Финальная визуализация после обучения

Запустите эту ячейку после завершения обучения для финальных графиков.


In [ ]:
# Финальная загрузка и визуализация (используем текущий файл, если он определен)
# Используем run_log_dir, если он определен, иначе fallback на TRAINING_CONFIG['log_dir']
log_dir_to_use = run_log_dir if 'run_log_dir' in globals() else TRAINING_CONFIG['log_dir']

if 'current_metrics_file' in globals() and current_metrics_file and Path(current_metrics_file).exists():
    df = load_metrics_from_csv(log_dir_to_use, metrics_file=current_metrics_file)
else:
    df = load_metrics_from_csv(log_dir_to_use)

if not df.empty:
    print("=" * 80)
    print("Финальные результаты обучения")
    print("=" * 80)
    print(f"Всего эпизодов: {len(df)}")
    print(f"Последний эпизод: {df['step'].max()}")
    print()
    
    # Показываем статистику
    if 'win_rate' in df.columns:
        print("Win Rate:")
        print(f"  Среднее: {df['win_rate'].mean():.2%}")
        print(f"  Последнее: {df['win_rate'].iloc[-1]:.2%}")
        print(f"  Максимум: {df['win_rate'].max():.2%}")
        print()
    
    if 'train_loss' in df.columns:
        print("Training Loss:")
        print(f"  Среднее: {df['train_loss'].mean():.4f}")
        print(f"  Последнее: {df['train_loss'].iloc[-1]:.4f}")
        print(f"  Минимум: {df['train_loss'].min():.4f}")
        print()
    
    # Строим финальные графики
    plot_win_rate_comparison(df)
    plot_metrics(df, PLOT_CONFIG['metrics_to_plot'])
    
    # Дополнительные графики
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Win rate
    if 'win_rate' in df.columns:
        axes[0, 0].plot(df['step'], df['win_rate'], label='Win Rate', color='green')
        axes[0, 0].set_title('Win Rate')
        axes[0, 0].set_xlabel('Episode')
        axes[0, 0].set_ylabel('Rate')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].set_ylim([0, 1])
    
    # Training loss
    if 'train_loss' in df.columns:
        axes[0, 1].plot(df['step'], df['train_loss'], label='Loss', color='red')
        axes[0, 1].set_title('Training Loss')
        axes[0, 1].set_xlabel('Episode')
        axes[0, 1].set_ylabel('Loss')
        axes[0, 1].grid(True, alpha=0.3)
    
    # Episode reward
    if 'episode_reward' in df.columns:
        axes[1, 0].plot(df['step'], df['episode_reward'], label='Reward', color='blue')
        axes[1, 0].set_title('Episode Reward')
        axes[1, 0].set_xlabel('Episode')
        axes[1, 0].set_ylabel('Reward')
        axes[1, 0].grid(True, alpha=0.3)
    
    # Epsilon
    if 'epsilon' in df.columns:
        axes[1, 1].plot(df['step'], df['epsilon'], label='Epsilon', color='orange')
        axes[1, 1].set_title('Epsilon (Exploration Rate)')
        axes[1, 1].set_xlabel('Episode')
        axes[1, 1].set_ylabel('Epsilon')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Визуализация завершена!")
else:
    print("❌ Нет данных для визуализации. Убедитесь, что обучение было запущено.")


In [ ]:
# Переимпорт модуля compare_checkpoints для применения изменений
# Выполните эту ячейку, если получили ошибку про отсутствие параметра 'epsilon'
import importlib
import src.training.compare_checkpoints
importlib.reload(src.training.compare_checkpoints)
print("✅ Модуль compare_checkpoints перезагружен!")


## Оценка обученной модели

Оцените обученную модель против разных оппонентов.


## Сравнение чекпоинтов друг с другом

Сравните несколько чекпоинтов в round-robin турнире. Каждая модель сыграет против каждой другой.


In [ ]:
# Сравнение чекпоинтов друг с другом
from src.training.compare_checkpoints import compare_checkpoints, plot_comparison_results, print_comparison_results
from pathlib import Path
import glob

# Настройки сравнения
COMPARISON_CONFIG = {
    "model_type": "dqn",  # "dqn" или "qlearning"
    "num_games_per_match": 100,  # Количество игр в каждом матче
    "epsilon": 0.01,  # 1% рандома (0.01 = 1% случайных ходов)
    
    "seed": 42,
    "device": None,  # None для auto-detect, или "cuda" или "cpu"
}

# Способы указать чекпоинты:
# 1. Указать конкретные пути к чекпоинтам
CHECKPOINT_PATHS = [
    # Примеры:
     "../data/checkpoints/dqn_20251109_180811/dqn_episode_5000.pt",
     "../data/checkpoints/dqn_20251109_180811/dqn_episode_10000.pt",
     "../data/checkpoints/dqn_20251109_180811/dqn_episode_15000.pt",
     "../data/checkpoints/dqn_20251109_180811/dqn_episode_20000.pt",
     "../data/checkpoints/dqn_20251109_180811/dqn_episode_25000.pt",
     "../data/checkpoints/dqn_20251109_180811/dqn_episode_30000.pt",
     "../data/checkpoints/dqn_20251109_180811/dqn_episode_35000.pt",
     "../data/checkpoints/dqn_20251109_180811/dqn_episode_40000.pt",
   #   "../data/checkpoints/dqn_20251109_000844/dqn_episode_40000.pt",
   #  "../data/checkpoints/dqn_20251109_000844/dqn_episode_7000.pt",
    # "../data/checkpoints/dqn_20251109_000844/dqn_episode_8000.pt",
   #  "../data/checkpoints/dqn_20251108_233748/dqn_episode_10000.pt",
   #  "../data/checkpoints/dqn_20251108_233748/dqn_episode_13000.pt",
     # "../data/checkpoints/dqn_20251108_195301/dqn_episode_27000.pt"
     
]

# 2. Или автоматически найти все чекпоинты в папке
# Раскомментируйте следующие строки для автоматического поиска:
# CHECKPOINT_DIR = "data/checkpoints/dqn_20240101_120000"  # Укажите папку с чекпоинтами
# CHECKPOINT_PATHS = sorted(glob.glob(f"{CHECKPOINT_DIR}/*.pt"))

# 3. Или найти все чекпоинты по паттерну
# CHECKPOINT_PATHS = sorted(glob.glob("data/checkpoints/**/dqn_episode_*.pt", recursive=True))

if not CHECKPOINT_PATHS:
    print("⚠️  Укажите пути к чекпоинтам в CHECKPOINT_PATHS!")
    print("Или раскомментируйте код для автоматического поиска чекпоинтов.")
else:
    print(f"📋 Найдено {len(CHECKPOINT_PATHS)} чекпоинтов для сравнения:")
    for i, path in enumerate(CHECKPOINT_PATHS, 1):
        print(f"  {i}. {path}")
    
    print(f"\n⚙️  Настройки:")
    print(f"  Тип модели: {COMPARISON_CONFIG['model_type']}")
    print(f"  Игр в матче: {COMPARISON_CONFIG['num_games_per_match']}")
    print(f"  Epsilon (рандом): {COMPARISON_CONFIG['epsilon']:.1%} ({COMPARISON_CONFIG['epsilon']*100:.1f}% случайных ходов)")
    print(f"  Seed: {COMPARISON_CONFIG['seed']}")
    
    # Запускаем сравнение
    print(f"\n🚀 Запуск сравнения...")
    print("=" * 80)
    
    results = compare_checkpoints(
        checkpoint_paths=CHECKPOINT_PATHS,
        model_type=COMPARISON_CONFIG["model_type"],
        num_games_per_match=COMPARISON_CONFIG["num_games_per_match"],
        epsilon=COMPARISON_CONFIG["epsilon"],
        device=COMPARISON_CONFIG["device"],
        seed=COMPARISON_CONFIG["seed"],
        reward_config=RewardConfig,
        use_epsilon=True,
        verbose=True,
    )
    
    # Выводим результаты
    print_comparison_results(results)
    
    # Визуализируем результаты
    print("\n📊 Визуализация результатов...")
    plot_comparison_results(results)


In [ ]:
import glob
import os
import matplotlib.pyplot as plt
from src.training.eval_model import evaluate_model_against_opponents

def evaluate_checkpoints_and_plot(
    checkpoint_dir="../data/checkpoints/dqn_20251109_102818",
    checkpoint_pattern="*.pt",
    model_type="dqn",
    opponents=("random", "heuristic", "smart_heuristic"),
    num_episodes=200,
    seed=42,
    epsilon=0.01
):
    """
    Оценивает все чекпоинты в папке против заданных оппонентов и строит график winrate.
    """
    # Собираем список чекпоинтов и сортируем его (по номеру эпизода, если возможно)
    pattern = os.path.join(checkpoint_dir, checkpoint_pattern)
    def extract_version(path):
        import re
        filename = os.path.basename(path)
        m = re.search(r"(\d+)", filename)
        return int(m.group(1)) if m else 0

    checkpoint_paths = sorted(glob.glob(pattern), key=extract_version)

    if not checkpoint_paths:
        print("❌ Не найдены чекпоинты по пути:", pattern)
        return

    print(f"🔍 Найдено {len(checkpoint_paths)} чекпоинтов")
    print(f"Оценка против {opponents} с epsilon={epsilon:.2f} ({epsilon*100:.1f}% random) на {num_episodes} партиях")
    print("{:<25} {:>8} {:>12} {:>12} {:>12}".format("Checkpoint", "Version", "Random", "Heuristic", "SmartHeur"))

    versions = []
    winrates_by_opponent = {opp: [] for opp in opponents}
    checkpoint_names = []

    for i, path in enumerate(checkpoint_paths, 1):
        version = extract_version(path)
        print(f"[{i}/{len(checkpoint_paths)}] {os.path.basename(path)}...", end=" ", flush=True)

        results = evaluate_model_against_opponents(
            model_path=path,
            model_type=model_type,
            opponents=opponents,
            num_episodes=num_episodes,
            seed=seed,
            use_epsilon=True,
        )

        # Вручную устанавливаем epsilon (для DQN сделает эффект если агент поддерживает)
        try:
            from src.agents import QLearningAgent, DQNAgent
            if model_type == "qlearning":
                agent = QLearningAgent(seed=seed)
                agent.load(path)
            else:
                agent = DQNAgent(rows=6, cols=7, seed=seed)
                agent.load(path)
            if hasattr(agent, 'epsilon'):
                agent.epsilon = epsilon
        except Exception:
            pass

        versions.append(version)
        checkpoint_names.append(os.path.basename(path))
        winrates = []
        for opp in opponents:
            win_rate = results[opp]["win_rate"]
            winrates.append(win_rate)
            winrates_by_opponent[opp].append(win_rate)
        print("{:<25} {:>8} {:>12.2%} {:>12.2%} {:>12.2%}".format(
            os.path.basename(path),
            version,
            winrates[0], winrates[1], winrates[2]
        ))

    # График после завершения
    plt.figure(figsize=(14, 6))
    for opp in opponents:
        plt.plot(versions, winrates_by_opponent[opp], marker='o', label=opp)
    plt.title(f"Win rate vs different opponents (epsilon={epsilon:.2f}, {num_episodes} игр)")
    plt.xlabel("Checkpoint version")
    plt.ylabel("Win rate")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.legend(title="Opponent")
    plt.tight_layout()
    plt.show()


In [ ]:
evaluate_checkpoints_and_plot(checkpoint_dir="../data/checkpoints/dqn_20251109_180811", use_epsilon=False)

In [ ]:
evaluate_checkpoints_and_plot(checkpoint_dir="../data/checkpoints/dqn_20251109_193645")

In [ ]:
evaluate_checkpoints_and_plot(checkpoint_dir="../data/checkpoints/dqn_20251109_170952")


In [ ]:
evaluate_checkpoints_and_plot(checkpoint_dir="../data/checkpoints/dqn_20251109_110859")